<a href="https://colab.research.google.com/github/Ankitkumar1015/pre-trained-models/blob/main/inceptionv3_google_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models
from torch.optim import lr_scheduler
import torch.optim as optim
import time

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms for train and test sets
train_transforms = transforms.Compose([
    transforms.Resize(330),
    transforms.CenterCrop(299),  # Input size required for Inception v3
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(330),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the datasets
data_dir = '/content/drive/MyDrive/Ml/gender. google - Copy'  # Example path from Google Drive

train_dataset = datasets.ImageFolder(root=f'{data_dir}/train', transform=train_transforms)
test_dataset = datasets.ImageFolder(root=f'{data_dir}/test', transform=test_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pretrained Inception v3 model
model = models.inception_v3(pretrained=True)

# Modify the last fully connected layer for binary classification (male/female)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: male, female

# Set the model to training mode and move to GPU if available
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Step learning rate decay
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training loop
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and evaluation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = test_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)

                    # If in training phase, use both outputs (main and auxiliary)
                    if phase == 'train':
                        outputs, aux_outputs = outputs
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4 * loss2  # Combine main and auxiliary losses
                    else:
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Track loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Update learning rate scheduler
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

# Train the model
train_model(model, criterion, optimizer, scheduler, num_epochs=10)

# Evaluate model accuracy on test set
model.eval()  # Set model to evaluation mode

correct = 0
total = 0

with torch.no_grad():  # No need to calculate gradients for evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 135MB/s] 


Epoch 1/10
----------
train Loss: 2.2111 Acc: 0.6533
val Loss: 0.4209 Acc: 0.8643
Epoch 2/10
----------
train Loss: 0.4202 Acc: 0.8978
val Loss: 0.2581 Acc: 0.8857
Epoch 3/10
----------
train Loss: 0.2180 Acc: 0.9444
val Loss: 0.1871 Acc: 0.9000
Epoch 4/10
----------
train Loss: 0.1400 Acc: 0.9667
val Loss: 0.1718 Acc: 0.9286
Epoch 5/10
----------
train Loss: 0.0911 Acc: 0.9811
val Loss: 0.2175 Acc: 0.9000
Epoch 6/10
----------
train Loss: 0.1073 Acc: 0.9778
val Loss: 0.2054 Acc: 0.9071
Epoch 7/10
----------
